In [1]:
import re
import yaml
import os

def extract_dram_utilized_capacity(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    in_dram = False
    current_section = None
    intput = 0
    output = 0
    weight = 0
    
    for line in lines:
        line = line.strip()

        # Detect the start of DRAM level
        if line.startswith("=== DRAM ==="):
            in_dram = True
            continue

        # Exit DRAM parsing if another level starts
        if in_dram and line.startswith("===") and not line.startswith("=== DRAM ==="):
            break

        # Identify current section
        if line.startswith("Inputs:"):
            current_section = 'input'
        elif line.startswith("Outputs:"):
            current_section = 'output'
        elif line.startswith("Weights:"):
            current_section = 'weight'

        # Extract Utilized capacity
        if in_dram and "Utilized capacity" in line:
            match = re.search(r'Utilized capacity\s+:\s+([0-9.]+)', line)
            if match and current_section:
                value = float(match.group(1))
                if current_section == 'input':
                    intput = value
                elif current_section == 'output':
                    output = value
                elif current_section == 'weight':
                    weight = value
                
    return [intput, output, weight]

In [2]:
def extract_total_chip_area(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    total_area = 0.0
    for line in lines:
        line = line.strip()
        if line.startswith("Area") and "um^2" in line:
            match = re.search(r'Area\s+\(total\)?\s*:\s*([0-9.]+)', line)
            if not match:
                match = re.search(r'Area\s*:\s*([0-9.]+)', line)
            if match:
                total_area += float(match.group(1))
    return total_area

In [3]:
def extract_total_energy(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line.startswith("Energy:") and "uJ" in line:
            match = re.search(r'Energy:\s+([0-9.]+)\s*uJ', line)
            if match:
                return float(match.group(1))
    return None

In [4]:
def extract_total_cycles(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("Cycles:"):
                match = re.search(r"Cycles:\s+([0-9]+)", line)
                if match:
                    return int(match.group(1))
    return None

In [5]:
def search_directories(path):
    directories = []
    for dirpath, dirnames, filenames in os.walk(path):
        directories.append(dirpath)
    return directories

In [6]:
def save_to_yaml(output_path, dram_caps, area, energy, cycles):
    result = {
        'dram_utilized_capacity': dram_caps,
        'total_chip_area_um2': area,
        'total_energy_uJ': energy,
        'total_cycles': cycles
    }

    with open(output_path, 'w') as yaml_file:
        yaml.dump(result, yaml_file, default_flow_style=False)

In [7]:
if __name__ == '__main__':
    tl_output_path = '/home/workspace/1_run_timeloop/outputs/'
    tl_file_name = 'timeloop-mapper.stats.txt'
    output_yaml = 'timeloop_summary.yaml'
    
    layer_dirs = search_directories(tl_output_path)
    dram_weight_caps = []
    energy = []
    cycles = []
    dram_inout_max = 0
    chip_area_max = 0
    
    for layer_dir in layer_dirs:
        if 'eyeriss_like' in layer_dir.split('/')[-1]:
            file_path = os.path.join(layer_dir, tl_file_name)
            
            dram_caps = extract_dram_utilized_capacity(file_path)
            if (dram_caps[0]+dram_caps[1]) > dram_inout_max:
                dram_inout_max = dram_caps[0]+dram_caps[1]
            dram_weight_caps.append(dram_caps[2])
            
            chip_area = extract_total_chip_area(file_path)
            if chip_area > chip_area_max:
                chip_area_max = chip_area
                
            energy.append(extract_total_energy(file_path))
            cycles.append(extract_total_cycles(file_path))
    
    total_dram_caps = dram_inout_max + sum(dram_weight_caps)
    total_chip_area = chip_area_max
    total_energy = sum(energy)
    total_cycles = sum(cycles)

    
    print("DRAM Specifications:")
    print(f"Total Capacity  : {total_dram_caps} Byte")
    print(f"Total Chip Area : {total_chip_area:.2f} um^2")
    print(f"Total Energy    : {total_energy:.2f} uJ")
    print(f"Total Cycles    : {total_cycles} cycles")

    save_to_yaml(output_yaml, total_dram_caps, total_chip_area, total_energy, total_cycles)
    print(f"\nYAML saved to: {output_yaml}")

DRAM Specifications:
Total Capacity  : 368547072.0 Byte
Total Chip Area : 238279.12 um^2
Total Energy    : 291592.94 uJ
Total Cycles    : 2754609152 cycles

YAML saved to: timeloop_summary.yaml
